In [1]:
import os
import torch

In [2]:
ssd_checkpoint_file = "/home/andrew/Development/wildfire/WIT-Asset-Detection/ssd_logs/2023_03_01__13_42_11/andrew_desktop_HIT+WIT_550_checkpoint_ssd300_state_dict.pt"
output_file_name = (
    os.path.splitext(os.path.basename(ssd_checkpoint_file))[0] + ".torchscript"
)
print(f"{output_file_name=}")

output_file_name='andrew_desktop_HIT+WIT_550_checkpoint_ssd300_state_dict.torchscript'


In [3]:
checkpoint = torch.load(ssd_checkpoint_file)
start_epoch = checkpoint["epoch"] + 1
print("\nLoaded checkpoint from epoch %d.\n" % start_epoch)


from ssd_model import SSD300
from ssd_utils import label_map

n_classes = len(label_map)
model = SSD300(n_classes=n_classes)

model.load_state_dict(checkpoint["model"])
model.eval()


Loaded checkpoint from epoch 551.


Loaded base model.

tensor([[0.0132, 0.0132, 0.1000, 0.1000],
        [0.0132, 0.0132, 0.1414, 0.1414],
        [0.0132, 0.0132, 0.1414, 0.0707],
        ...,
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 0.6364],
        [0.5000, 0.5000, 0.6364, 1.0000]])


SSD300(
  (base): VGGBase(
    (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv4_1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv4_2): Conv2d(512, 512, kernel_size=(3, 3), 

In [4]:
torch.set_printoptions(profile="full")

In [5]:
import PIL.Image

SSD_c, SSD_h, SSD_w = 3, 300, 300
sample_image_input = torch.zeros(1, SSD_c, SSD_h, SSD_w)

# image_file_path = "/home/andrew/Downloads/thermal.png"
# image_file_path = "/home/andrew/Downloads/black.png"
image_file_path = "/home/andrew/Downloads/gray.png"
sample_image_input = PIL.Image.open(image_file_path)


from torchvision.transforms.functional import to_tensor, normalize, resize

sample_tensor = to_tensor(sample_image_input)
sample_tensor = resize(sample_tensor, (SSD_h, SSD_w))
sample_tensor = normalize(
    sample_tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
).unsqueeze(0)
sample_tensor.shape

module = torch.jit.load(
    "/home/andrew/Development/ipp_ws/src/ipp_computer_vision/object_detection/config/wildfire/torchscript_models/andrew_desktop_HIT+WIT_550_checkpoint_ssd300_state_dict.torchscript"
)
module.eval()
predicted_locs, predicted_scores = module(sample_tensor)
det_boxes, det_labels, det_scores = model.detect_objects(
    predicted_locs, predicted_scores, min_score=0.2, max_overlap=0.5, top_k=200
)

/home/andrew/.mambaforge/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [6]:
data = sample_tensor.cuda()

torch.onnx.export(
    model.cuda().eval(),
    data,
    os.path.splitext(os.path.basename(ssd_checkpoint_file))[0] + ".onnx",
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
)

================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
traced_script_module = torch.jit.trace(model, sample_image_input)

Check the output make sure it's the same

In [ ]:
traced_output = traced_script_module(sample_image_input)
print(traced_output[0].shape)
print(traced_output[1].shape)

In [ ]:
actual_output = model(sample_image_input)

In [ ]:
traced_output[0]

In [ ]:
print("Norm=", torch.norm(traced_output[0] - actual_output[0]))

In [ ]:
traced_script_module.save(output_file_name)
print("Done! Output to:", output_file_name)

In [ ]:
loaded_module = torch.jit.load(output_file_name)

In [ ]:
loaded_module(sample_tensor)[0]